In [1]:
import pandas as pd
from sqlalchemy import create_engine

import psycopg2

from config import password

In [3]:
tornado_file = "Historical_Tornado_Tracks.csv"
tornado_df = pd.read_csv(tornado_file)
tornado_df.head()

,FID,OM,YR,MO,DY,DATE,TIME,TZ,ST,STF,...,FAT,LOSS,CLOSS,SLAT,SLON,ELAT,ELON,LEN,WID,Shape_Leng
0,2001,141,2002,6,5,2002-06-05,16:45:00,3,FL,12,...,0,0.000,0.0,28.08,-82.78,28.08,-82.78,0.1,20,0.000000
1,2002,142,2002,6,8,2002-06-08,13:40:00,3,FL,12,...,0,0.500,0.0,27.73,-82.73,27.72,-82.75,1.0,40,0.022361
2,2003,145,2002,6,17,2002-06-17,16:40:00,3,FL,12,...,0,0.000,0.0,27.48,-82.72,27.48,-82.72,0.1,20,0.000000
3,2004,187,2002,7,11,2002-07-11,12:30:00,3,FL,12,...,0,0.020,0.0,27.77,-82.77,27.77,-82.77,0.5,40,0.000000
4,2005,188,2002,7,12,2002-07-12,06:30:00,3,FL,12,...,0,0.001,0.0,27.92,-82.80,27.92,-82.80,0.5,40,0.000000


In [4]:
# Create a filtered dataframe from specific columns
tornado_cols = ["YR", "DATE", "MAG", "SLAT", "SLON"]
tornado_transformed= tornado_df[tornado_cols].copy()

# Rename the column headers
tornado_transformed = tornado_transformed.rename(columns={"YR": "year",
                                                         "DATE": "date",
                                                         "MAG": "mag",
                                                         "SLAT": "lat",
                                                         "SLON": "lon"})

tornado_transformed.head()

,year,date,mag,lat,lon
0,2002,2002-06-05,0,28.08,-82.78
1,2002,2002-06-08,0,27.73,-82.73
2,2002,2002-06-17,0,27.48,-82.72
3,2002,2002-07-11,0,27.77,-82.77
4,2002,2002-07-12,0,27.92,-82.80


In [14]:
# Filter data by year to only show 2011 through 2013

tornado_filtered = tornado_transformed.loc[(tornado_transformed['year'] >= 2011) & (tornado_transformed['year'] <= 2013)]

tornado_filtered.head()

,year,date,mag,lat,lon
723,2011,2011-01-09,1,27.7107,-97.9663
733,2011,2011-03-31,1,27.8716,-82.8509
736,2011,2011-03-31,1,27.8983,-82.6935
738,2011,2011-05-13,0,29.2061,-90.0433
741,2011,2011-05-13,0,29.3500,-89.5300


In [6]:
earthquake_file = "earthquakes_2011-2013.csv"
earthquake_df = pd.read_csv(earthquake_file)
earthquake_df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2013-10-19T17:54:54.700Z,26.091300,-110.3209,9.450,6.60,mww,NaN,44.0,1.98000,1.10,...,2020-07-10T18:20:12.984Z,"98 km SW of Etchoropo, Mexico",earthquake,NaN,3.00,NaN,NaN,reviewed,us,us
1,2013-01-13T16:28:26.220Z,25.891000,-110.1030,10.000,5.70,mww,429.0,60.1,NaN,1.03,...,2020-09-21T18:36:56.779Z,"76 km W of Las Grullas Margen Derecha, Mexico",earthquake,NaN,NaN,NaN,NaN,reviewed,us,us
2,2013-05-24T03:47:08.180Z,40.191833,-121.0595,7.968,5.69,mw,32.0,69.0,0.08649,0.08,...,2020-07-10T14:24:39.433Z,"2 km NNE of Canyondam, California",earthquake,0.26,0.68,NaN,5.0,reviewed,nc,nc
3,2013-01-13T17:50:13.000Z,25.773000,-110.1260,10.000,5.40,mwc,268.0,123.9,NaN,1.40,...,2015-03-24T02:25:53.541Z,"79 km W of Las Grullas Margen Derecha, Mexico",earthquake,NaN,NaN,NaN,NaN,reviewed,us,gcmt
4,2013-09-21T12:16:12.170Z,27.801300,-105.6794,10.000,5.40,mww,NaN,41.0,0.86200,0.54,...,2015-01-29T21:29:02.933Z,"19 km WSW of Naica, Mexico",earthquake,NaN,1.40,NaN,NaN,reviewed,us,us


In [7]:
# Create a filtered dataframe from specific columns
earthquake_cols = ["time", "latitude", "longitude", "mag"]
earthquake_transformed= earthquake_df[earthquake_cols].copy()

earthquake_transformed.head()

,time,latitude,longitude,mag
0,2013-10-19T17:54:54.700Z,26.091300,-110.3209,6.60
1,2013-01-13T16:28:26.220Z,25.891000,-110.1030,5.70
2,2013-05-24T03:47:08.180Z,40.191833,-121.0595,5.69
3,2013-01-13T17:50:13.000Z,25.773000,-110.1260,5.40
4,2013-09-21T12:16:12.170Z,27.801300,-105.6794,5.40


In [8]:
#create date only from time column
time_split = earthquake_transformed['time'].str.split('T',expand=True)

earthquake_transformed['date'] = time_split[0]
earthquake_transformed.head()

,time,latitude,longitude,mag,date
0,2013-10-19T17:54:54.700Z,26.091300,-110.3209,6.60,2013-10-19
1,2013-01-13T16:28:26.220Z,25.891000,-110.1030,5.70,2013-01-13
2,2013-05-24T03:47:08.180Z,40.191833,-121.0595,5.69,2013-05-24
3,2013-01-13T17:50:13.000Z,25.773000,-110.1260,5.40,2013-01-13
4,2013-09-21T12:16:12.170Z,27.801300,-105.6794,5.40,2013-09-21


In [9]:
#extracted year only from time and dropped time column

time_split2 = earthquake_transformed['time'].str.split('-',expand=True)
earthquake_transformed['year'] = time_split2[0]

earthquake_transformed.drop(columns=['time'], inplace=True)
earthquake_transformed.head()

,latitude,longitude,mag,date,year
0,26.091300,-110.3209,6.60,2013-10-19,2013
1,25.891000,-110.1030,5.70,2013-01-13,2013
2,40.191833,-121.0595,5.69,2013-05-24,2013
3,25.773000,-110.1260,5.40,2013-01-13,2013
4,27.801300,-105.6794,5.40,2013-09-21,2013


In [13]:
# Rename the column headers
earthquake_transformed = earthquake_transformed.rename(columns={
                                                          "latitude": "lat",
                                                          "longitude": "lon"})

earthquake_transformed.head()

,lat,lon,mag,date,year
0,26.091300,-110.3209,6.60,2013-10-19,2013
1,25.891000,-110.1030,5.70,2013-01-13,2013
2,40.191833,-121.0595,5.69,2013-05-24,2013
3,25.773000,-110.1260,5.40,2013-01-13,2013
4,27.801300,-105.6794,5.40,2013-09-21,2013


In [15]:
tornado_filtered.to_csv('../static/data/tornado_cleaned.csv')
earthquake_transformed.to_csv('../static/data/earthquake_cleaned.csv')

In [30]:


connection_string = f"postgres:{password}@localhost:5432/disaster_db"
engine = create_engine(f'postgresql://{connection_string}')

In [31]:
tornado_filtered.to_sql(name='tornado', con=engine, if_exists='append', index=True)

In [32]:
earthquake_transformed.to_sql(name='earthquake', con=engine, if_exists='append', index=True)